In [56]:
import xml.etree.cElementTree as ET
import pprint
import collections as co
import re
import codecs
import csv
import cerberus
import sqlite3

In [57]:
OSM_PATH = "mumbai_india.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

schema = {
    'node': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'lat': {'required': True, 'type': 'float', 'coerce': float},
            'lon': {'required': True, 'type': 'float', 'coerce': float},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'node_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    },
    'way': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'way_nodes': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'node_id': {'required': True, 'type': 'integer', 'coerce': int},
                'position': {'required': True, 'type': 'integer', 'coerce': int}
            }
        }
    },
    'way_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    }
}

In [58]:
def validate_element(element, validator, schema=schema):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))
        
class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)

In [59]:
unwanted_cities_list = ['Ambernath','Andheri','Andheri West','Bandra, Mumbai','Chembur, Mumbai',\
                        'Ghatkopar West, Mumbai','KURLA','MUMBAI','Mulund','Mulund (East)','Mulund (West)',\
                        'Mulund(West)','Mumabi','Mumbai,Kurla(East)','cheeta camp, mumbai','mUMBAI','mumbai']

In [60]:
def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []
    way_nodes = []
    way_tags = []
    
    if element.tag == "node":
        for i in element.attrib:
            if i in node_attr_fields:
                node_attribs[i]=element.attrib[i]
        for child in element:
            temp = {}
            temp["id"] = element.attrib["id"]
            temp["value"] = child.attrib["v"]
            k = child.attrib["k"].split(":")
            if len(k)>1:
                temp["type"] = k[0]
                del k[0]
                temp["key"] = ":".join(k)
            else:
                temp["type"] = "regular"
                temp["key"] = child.attrib["k"]
            
            tags.append(temp)
    
    if element.tag == "way":
        for i in element.attrib:
            if i in way_attr_fields:
                way_attribs[i]=element.attrib[i]
        num = 0
        for child in element:
            if child.tag == "nd":
                temp1 = {}
                temp1["id"] = element.attrib["id"]
                temp1["node_id"] = child.attrib["ref"]
                temp1["position"] = num
                num = num+1
                way_nodes.append(temp1)
            if child.tag == "tag":
                temp2 = {}
                temp2["id"] = element.attrib["id"]
                temp2["value"] = child.attrib["v"]
                k = child.attrib["k"].split(":")
                if len(k)>1:
                    temp2["type"] = k[0]
                    del k[0]
                    temp2["key"] = ":".join(k)
                else:
                    temp2["type"] = "regular"
                    temp2["key"] = child.attrib["k"]
                way_tags.append(temp2)
            
    if element.tag == 'node':
        #print {'node': node_attribs, 'node_tags': tags}
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        #print {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': way_tags}
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': way_tags}

In [61]:
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        for child in elem:
            """rectifying for postal code"""
            if child.tag == "tag" and (child.attrib["k"] == "postal_code" or child.attrib["k"] == "addr:postcode"):
                child.attrib["k"] = "addr:postcode"
                temp = child.attrib["v"].split()
                if len(temp)>1:
                    child.attrib["v"] = "".join(temp)
                    
            """rectifying for city name"""
            if child.tag=="tag" and child.attrib["k"] == "addr:city" and (child.attrib["v"] in unwanted_cities_list):
                child.attrib["v"] = "Mumbai"
                    
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

In [62]:
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            #print el
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])

In [63]:
if __name__ == '__main__':
    process_map(OSM_PATH, validate=True)

In [65]:
db = sqlite3.connect("Mumbai")
c = db.cursor()
query1 = '''CREATE TABLE IF NOT EXISTS nodes(
            id real PRIMARY KEY,
            lat real NOT NULL,
            lon real NOT NULL,
            user real NOT NULL,
            uid real NOT NULL,
            version real NOT NULL,
            changeset real NOT NULL,
            timestamp text NOT NULL);
        '''

query2 = '''CREATE TABLE IF NOT EXISTS nodes_tags(
            id real NOT NULL,
            key text NOT NULL,
            value text NOT NULL,
            type text NOT NULL,
            foreign key (id) references nodes (id));
        '''

query3 = '''CREATE TABLE IF NOT EXISTS ways(
            id real PRIMARY KEY,
            user real NOT NULL,
            uid real NOT NULL,
            version real NOT NULL,
            changeset real NOT NULL,
            timestamp text NOT NULL);
        '''

query4 = '''CREATE TABLE IF NOT EXISTS ways_tags(
            id real NOT NULL,
            key text NOT NULL,
            value text NOT NULL,
            type text NOT NULL,
            foreign key (id) references ways (id));
        '''

query5 = '''CREATE TABLE IF NOT EXISTS ways_nodes(
            id real NOT NULL,
            node_id real NOT NULL,
            position text NOT NULL,
            foreign key (id) references ways (id));
        '''
c.execute(query1)
c.execute(query2)
c.execute(query3)
c.execute(query4)
c.execute(query5)

### KINDLY NOTE:
##### The steps required to insert data from csv to sqlite table was carried out in the sqlite3 using the following code

```sql
   sqlite3
   .open Mumbai
   .mode csv
   .import nodes.csv nodes
   .import nodes_tags.csv nodes_tags
   .import ways.csv ways
   .import ways_nodes.csv ways_nodes
   .import ways_tags.csv ways_tags
```